Autor: Esteban Suárez Calvo

In [14]:
from IPython.display import clear_output
from sklearn.svm import SVC
from qiskit.circuit.library import n_local, zz_feature_map
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import COBYLA
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# CCAM - Práctica 2: Redes Neuronales Cuánticas

## 1.Preprocesado

### Normalización del dataset

Normalizamos el dataset utilizando la normalización MinMax.

In [2]:
data = load_breast_cancer()
X = data.data
y = data.target
X = MinMaxScaler().fit_transform(X)
print(f"Número de características = {X.shape[1]}")

Número de características = 30


### Reducción de dimensionalidad

Reducimos la dimensionalidad del dataset utilizando una reducción PCA. De no hacerlo, necesitaríamos un total de 30 qubits, pues tenemos 30 características, y cada una se representa con un qubit. El problema está en que, aunque 30 qubits pueden no parecer demasiados, puede suponer un costo computacional bastante elevado el tener que emularlos.

Sin embargo, reducir la dimensionalidad tiene también su parte negativa, y es que al reducirla, estas dimensiones colapsan en menos, lo que hace que las clases que, anteriormente, podrían ser separables, puedan ya no serlo, por lo que debemos de ser cuidadosos y no pasarnos al reducir las dimensiones.

In [3]:
X = PCA(n_components=10).fit_transform(X)
print(f"Nuevo número de características = {X.shape[1]}")

Nuevo número de características = 10


### Dividir el conjunto entre entrenamiento y test

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20)
print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de test: {len(X_test)}")

Datos de entrenamiento: 455
Datos de test: 114


### Clasificación con un SVM

Para comprobar que el problema es resoluble con la reducción de dimensionalidad que hemos aplicado antes, utilizamos un SVM

In [15]:
svc = SVC()
svc.fit(X_train, Y_train)

train_score = svc.score(X_train, Y_train)
test_score = svc.score(X_test, Y_test)

print(f"SVC en entrenamiento: {train_score}")
print(f"SVC en test: {test_score}")

SVC en entrenamiento: 0.9736263736263736
SVC en test: 1.0


## 2. Desarrolla una QNN para predecir la clase de cada caso

### Elegir un feature map

In [5]:
number_of_features = X.shape[1]
feature_map = zz_feature_map(number_of_features)

### Elegir un ansatz

In [6]:
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=2)

### Elegir un optimizador

In [7]:
optimizer = COBYLA(maxiter=30)

### Construir un VQC

In [11]:
loss_history = []


def callback_keras_style(weights, obj_func_eval):
    loss_history.append(obj_func_eval)

    iteration = len(loss_history)

    print(f"Iteración {iteration} - Loss: {obj_func_eval:.4f}")

In [9]:
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_keras_style,
)

## 3. Ejecutar el VQC con el conjunto de entrenamiento

In [10]:
vqc.fit(X_train, Y_train)

Iteración 1 - Loss (Coste): 0.9761
Iteración 2 - Loss (Coste): 0.9813
Iteración 3 - Loss (Coste): 0.9739
Iteración 4 - Loss (Coste): 0.9714
Iteración 5 - Loss (Coste): 0.9714
Iteración 6 - Loss (Coste): 0.9714
Iteración 7 - Loss (Coste): 0.9714
Iteración 8 - Loss (Coste): 0.9714
Iteración 9 - Loss (Coste): 0.9714
Iteración 10 - Loss (Coste): 0.9714
Iteración 11 - Loss (Coste): 0.9714
Iteración 12 - Loss (Coste): 0.9717
Iteración 13 - Loss (Coste): 0.9927
Iteración 14 - Loss (Coste): 0.9714
Iteración 15 - Loss (Coste): 0.9714
Iteración 16 - Loss (Coste): 0.9714
Iteración 17 - Loss (Coste): 0.9714
Iteración 18 - Loss (Coste): 0.9714
Iteración 19 - Loss (Coste): 0.9714
Iteración 20 - Loss (Coste): 0.9714
Iteración 21 - Loss (Coste): 0.9714
Iteración 22 - Loss (Coste): 0.9855
Iteración 23 - Loss (Coste): 0.9714
Iteración 24 - Loss (Coste): 0.9714
Iteración 25 - Loss (Coste): 0.9714
Iteración 26 - Loss (Coste): 0.9714
Iteración 27 - Loss (Coste): 0.9714
Iteración 28 - Loss (Coste): 0.9714
I

In [16]:
train_score = vqc.score(X_train, Y_train)
test_score = vqc.score(X_test, Y_test)

print(f"SVC en entrenamiento: {train_score}")
print(f"SVC en test: {test_score}")

SVC en entrenamiento: 0.6307692307692307
SVC en test: 0.5526315789473685
